In [26]:
from rdflib import Literal, Graph, BNode, RDF, Namespace
from random import randint
from enum import Enum

import os
import pickle
import sys


NUMBER_OF_SAMPLES = 1000
BASE_FILENAME = 'sample_'
EXTENSION = '.ttl'
DELIMITER = '/'
DIRNAME = 'generated_samples'
KEY_FILENAME = 'key.pkl'


class SecurityCondition(Enum):
    red = 0
    orange = 1
    yellow = 2
    green = 3
    
    def __str__(self):
        s = super(SecurityCondition, self).__str__()
        return "%s" % s.split('.')[1]


class DataFactory(object):
    
    GRAPH = 'graph'
    NAMESPACE = 'namespace'
    KEY = 'key'
    
    def __init__(self, graph, namespace, is_fuzzed=False, key=None):
        self.is_fuzzed = is_fuzzed
        self.graph = graph
        self.namespace = namespace
        self.key = key if key != None else {}
    
    def build(self):
        self.build_components()
        return {
            DataFactory.GRAPH: self.graph, 
            DataFactory.NAMESPACE: self.namespace, 
            DataFactory.KEY: self.key
        }
    

class HardDataFactory(DataFactory):
    
    def generate_random_heading(self):
        return randint(0, 180)

    def build_boats(self):
        heading = self.generate_random_heading() if self.is_fuzzed else 10
        
        id = self.namespace.id
        id_value = randint(0, sys.maxint)
        self.graph.add((id, RDF.type, Literal('id')))
        self.graph.add((id, Literal('id'), Literal(id_value)))
        
        boat = self.namespace.boat_1
        self.graph.add((boat, RDF.type, Literal('boat')))
        self.graph.add((boat, Literal('speed'), Literal(25)))
        self.graph.add((boat, Literal('heading'), Literal(str(heading))))  
        self.graph.add((boat, Literal('x'), Literal('2')))
        self.graph.add((boat, Literal('y'), Literal('1')))
                       
        boat = self.namespace.boat_2
        self.graph.add((boat, RDF.type, Literal('boat')))
        self.graph.add((boat, Literal('speed'), Literal(25)))
        self.graph.add((boat, Literal('heading'), Literal(str(heading))))  
        self.graph.add((boat, Literal('x'), Literal('2')))
        self.graph.add((boat, Literal('y'), Literal('2')))
        
        self.key[id_value] = True if heading > 45 and heading < 135 else False

    def build_target(self):
        ship = self.namespace.ship
        self.graph.add((ship, RDF.type, Literal('ship')))
        self.graph.add((ship, Literal('speed'), Literal(12)))
        self.graph.add((ship, Literal('heading'), Literal('10')))  
        self.graph.add((ship, Literal('x'), Literal('10')))
        self.graph.add((ship, Literal('y'), Literal('2')))
    
    def build_components(self):
        self.build_boats()
        self.build_target()
    
    
class SoftDataFactory(DataFactory):
    
    def build_security_condition(self):
        condition = SecurityCondition(randint(0, 3)) if self.is_fuzzed else SecurityCondition.red
        graphCondition = self.namespace.condition
        self.graph.add((graphCondition, Literal('SecurityCondition'), Literal(str(condition))))
        
    def build_components(self):
        self.build_security_condition()


graph_collection = []
key = {}
for i in range(0, NUMBER_OF_SAMPLES):

    hf = HardDataFactory(
        graph=Graph(), 
        namespace=Namespace('http://www.unm.edu/scenario/'),
        key=key,
        is_fuzzed=True
    )
    results = hf.build()

    sf = SoftDataFactory(
        graph=results[DataFactory.GRAPH], 
        namespace=results[DataFactory.NAMESPACE],
        key=results[DataFactory.KEY],
        is_fuzzed=True)
    results = sf.build()
    
    graph_collection.append(results[DataFactory.GRAPH])
    
os.mkdir(DIRNAME)
idx = 0
for g in graph_collection:
    filename = DIRNAME + DELIMITER + BASE_FILENAME + str(idx) + EXTENSION
    idx += 1
    with open(filename, 'w') as file:
        file.write(g.serialize(format='turtle'))
        
full_key_filename = DIRNAME + DELIMITER + KEY_FILENAME
with open(full_key_filename, 'wb') as file:
    pickle.dump(key, file)
    
print 'Files generated.'


Files generated.
